# Surprise models  

In [35]:
import pandas as pd
import numpy as np
import warnings
import os
import importlib
warnings.filterwarnings('ignore')


In [36]:
from google.colab import files
from google.colab import drive

In [38]:
# Mount Google Drive
drive.mount('/content/drive')
file_path= "/content/drive/MyDrive/prepro19.csv"
df=pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
df.head(5)

,Unnamed: 0,Product_name,Product_ID,Product_image,Price,User_ID,Rating,Product_description
0,0,HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,0030672120,https://www.gmt-sales.com/wp-content/uploads/2...,3565.0,A15YSF93XJUR94,4.0,no description for this product
1,1,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",0071480935,https://images-na.ssl-images-amazon.com/images...,3565.0,A2E7H1GZ49KT15,3.0,Latin rhythms that will get your kids singing ...
2,2,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",0071480935,https://images-na.ssl-images-amazon.com/images...,3565.0,A1OP7G76WA8IJ0,5.0,Latin rhythms that will get your kids singing ...
3,3,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",0071480935,https://images-na.ssl-images-amazon.com/images...,3565.0,A2AGDDGAV7XEHF,5.0,Latin rhythms that will get your kids singing ...
4,4,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",0071480935,https://images-na.ssl-images-amazon.com/images...,3565.0,AXRKS89VZWW4G,5.0,Latin rhythms that will get your kids singing ...


In [40]:
#checking the null values in each column
df.isnull().sum()

Unnamed: 0             0
Product_name           0
Product_ID             0
Product_image          0
Price                  0
User_ID                0
Rating                 0
Product_description    0
dtype: int64

In [41]:
!pip install surprise

In [42]:
#import surprise library for collebrative filtering
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [43]:
df.shape

(450232, 8)

In [44]:
#group by the product id that have more than 50 ratings
df_grouped=df.groupby('Product_ID').filter(lambda x:x['Rating'].count()>=50)


In [45]:
df_grouped.shape

(331521, 8)

In [46]:
#sampling the data and taking only 20k rows
electronics_data=df_grouped.sample(n=20000,ignore_index=True)

In [47]:
#drop Unnamed :0 column
electronics_data.drop('Unnamed: 0',axis=1,inplace=True)

In [48]:
electronics_data.shape

(20000, 7)

In [49]:
electronics_data.head(5)

,Product_name,Product_ID,Product_image,Price,User_ID,Rating,Product_description
0,Quicken Starter Edition 2011 - [Old Version],B003YJ5DSS,https://www.gmt-sales.com/wp-content/uploads/2...,3565.0,A10W02UXELJ05N,1.0,Simple Money Management. Organizes your bank a...
1,Mac OS X version 10.6.3 Snow Leopard (Mac comp...,B001AMHWP8,https://images-na.ssl-images-amazon.com/images...,3699.0,AQZH7YTWQPOBE,4.0,"Mac OS X 10.6.3 Snow Leopard, a simpler, more ..."
2,Microsoft Office 365 Personal | 1-year subscri...,B00IT6WQDQ,https://images-na.ssl-images-amazon.com/images...,6999.0,A3UO13JYVKDFIN,3.0,Office 365 comes fully loaded with the latest ...
3,Pure 3.0 - 3 User,B00BKWXNDI,https://images-na.ssl-images-amazon.com/images...,5988.0,A1DECEN4B3U34K,5.0,Product Description Pure 1303121 . Ultimate PC...
4,TurboTax Deluxe 2015 Federal + State Taxes - T...,B01637RFR4,https://www.gmt-sales.com/wp-content/uploads/2...,3565.0,A3EPJP49I4T0Q4,5.0,Get your taxes done right with TurboTax 2015.


In [50]:
electronics_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Product_name         20000 non-null  object 
 1   Product_ID           20000 non-null  object 
 2   Product_image        20000 non-null  object 
 3   Price                20000 non-null  float64
 4   User_ID              20000 non-null  object 
 5   Rating               20000 non-null  float64
 6   Product_description  20000 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.1+ MB


In [51]:
#drop the columns and saving just the 3 columns of product_id, user_id, rating
data = electronics_data.drop(['Product_name', 'Product_image', 'Price', 'Product_description'], axis=1)
data.head(5)

,Product_ID,User_ID,Rating
0,B003YJ5DSS,A10W02UXELJ05N,1.0
1,B001AMHWP8,AQZH7YTWQPOBE,4.0
2,B00IT6WQDQ,A3UO13JYVKDFIN,3.0
3,B00BKWXNDI,A1DECEN4B3U34K,5.0
4,B01637RFR4,A3EPJP49I4T0Q4,5.0


In [52]:
#Reading the dataset
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(data, reader)

In [53]:
#Splitting surprise the dataset into 70,30 ratio using train_test_split
trainset, testset = train_test_split(surprise_data, test_size=0.3,random_state=42)

In [54]:
# Number of users in the trainset
print(f"Number of users: {trainset.n_users}")

# Number of items in the trainset
print(f"Number of items: {trainset.n_items}")

# Number of ratings in the trainset
print(f"Number of ratings: {trainset.n_ratings}")

Number of users: 1361
Number of items: 13840
Number of ratings: 14000


In [55]:
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, CoClustering, SVD
predictions =[]
# Use user_based true/false to switch between user-based or item-based collaborative filtering
algo = KNNWithMeans(k=2, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo1 = KNNBasic(k=2, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo2 = KNNBaseline(k=2, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo3 = KNNWithZScore(k=2, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo4 = SVD()
algo5 = CoClustering()
#regrouper les modèles en une liste
score = [algo, algo1, algo2, algo3, algo4, algo5]
#boucle pour calculer l'erreru RMSE pour chaque modèle
for alg in score:
  alg.fit(trainset)
  alg.test(testset)
  test_pred=alg.test(testset)
  predictions.append(accuracy.rmse(test_pred ,verbose=True))
#make prediction using testset
print(predictions)
#choisir le modèle qui possède le min des erreurs
minValue = min(predictions)
index = predictions.index(minValue)
print(index)
alg = score[index]


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.5826
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.5835
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.5188
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.5826
RMSE: 1.5138
RMSE: 1.5831
[1.5826073025791556, 1.5834869427813996, 1.5187567291629847, 1.5826073025791556, 1.513769369659026, 1.5831402600495956]
4


In [56]:
# Get the dictionary of user ratings (user_id => [(item_inner_id, rating)])
user_ratings = trainset.ur

# Access user IDs along with their respective indices
for user_id, user_ratings_list in user_ratings.items():
    id_user = trainset.to_raw_uid(user_id)
    print(f"User ID: {user_id}, Index: {trainset.to_raw_uid(user_id)}")
    break
print(id_user)

User ID: 0, Index: B006T4SQG6
B006T4SQG6


In [57]:
# Get the dictionary of item ratings (item_id => [(user_inner_id, rating)])
item_ratings = trainset.ur

# Access item IDs along with their respective indices
for item_id, item_ratings_list in item_ratings.items():
    id_item = trainset.to_raw_uid(item_id)
    print(f"Item ID: {item_id}, Index: {id_item}")
    break
print(id_item)

Item ID: 0, Index: B006T4SQG6
B006T4SQG6


In [58]:
# Get the dictionary of user ratings (user_id => [(item_inner_id, rating)])
user_ratings = trainset.ur

# Access user IDs along with their respective ratings
for user_id, user_ratings_list in user_ratings.items():
    id_user = trainset.to_raw_uid(user_id)

    print(f"User ID: {id_user}, Ratings: {user_ratings_list[0][0]}")
    break

User ID: B006T4SQG6, Ratings: 0


In [59]:
import pickle

# Assuming 'algo' is your trained model
# Save the model to a file
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(alg, f)

# Load the saved model from the file
with open('trained_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [60]:
# Function to get recommended items for a specific user
def get_recommendations(algo, trainset, user_raw_id, n_recommendations):
    # Convert raw user ID to internal ID
    user_inner_id = trainset.to_inner_uid(user_raw_id)
    # Get all rated items for this user
    rated_items = set(x for x in trainset.ur[user_inner_id])
    #print(rated_items)
    # Create a list of all item IDs
    all_items = set(trainset.to_raw_iid(x) for x in range(trainset.n_items))
    # Get unrated items
    unrated_items = all_items - rated_items
    # Predict ratings for unrated items
    predictions = []
    for item in unrated_items:
        prediction = algo.predict(user_raw_id, item)
        predictions.append(prediction)
    # Sort predictions based on estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)
    # Return the top N recommendations
    return predictions[:n_recommendations]  # Return the top N recommendations as a list of tuples
# Example: Get top 10 recommendations for a specific user (raw ID)
user_raw_id = id_user  # Replace with the raw ID of the desired user
recommendations = get_recommendations(loaded_model, trainset, user_raw_id, 10)
# Display the top 10 recommended items for the user
for prediction in recommendations:
    print(f"Item: {prediction.uid}, Estimated Rating: {prediction.est:.2f}")


Item: B006T4SQG6, Estimated Rating: 4.90
Item: B006T4SQG6, Estimated Rating: 4.86
Item: B006T4SQG6, Estimated Rating: 4.85
Item: B006T4SQG6, Estimated Rating: 4.81
Item: B006T4SQG6, Estimated Rating: 4.81
Item: B006T4SQG6, Estimated Rating: 4.80
Item: B006T4SQG6, Estimated Rating: 4.80
Item: B006T4SQG6, Estimated Rating: 4.79
Item: B006T4SQG6, Estimated Rating: 4.79
Item: B006T4SQG6, Estimated Rating: 4.79
